#Non-Bass Way to do peak detect
####requires installation of pypeaks (https://github.com/gopalkoduri/pypeaks)

In [192]:
import pypeaks as pypk
import pickle
import sys
import os
import os.path as path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from matplotlib import colors


'''
data = pickle.load(file("./sample-histogram.pickle"))
#print data
hist = pypk.Data(data[0], data[1])
'''
#ji_intervals = pickle.load(file('./ji-intervals.pickle'))
#print ji_intervals

#pypk.Data.get_peaks??

data = pd.DataFrame.from_csv('p01_r01_f_2.txt', header=[0,1], sep="\t")
data.head(2)

Elapsed time,iltered
(seconds),(mV)
0.000,-0.115
0.002,-0.115


```
Signature: hist.get_peaks(method='slope', peak_amp_thresh=5e-05, 
                          valley_thresh=3e-05, intervals=None, lookahead=20, avg_interval=100)
Docstring:
This function expects SMOOTHED histogram. If you run it on a raw histogram,
there is a high chance that it returns no peaks.

method can be interval/slope/hybrid.
    The interval-based method simply steps through the whole histogram
    and pick up the local maxima in each interval, from which irrelevant
    peaks are filtered out by looking at the proportion of points on 
    either side of the detected peak in each interval, and by applying
    peal_amp_thresh and valley_thresh bounds.

    Slope approach uses, of course slope information, to find peaks, 
    which are then filtered by applying peal_amp_thresh and 
    valley_thresh bounds. 
    
    Hybrid approach combines the peaks obtained using slope method and
    interval-based approach. It retains the peaks/valleys from slope method
    if there should be a peak around the same region from each of the methods.

peak_amp_thresh is the minimum amplitude/height that a peak should have
in a normalized smoothed histogram, to be qualified as a peak. 
valley_thresh is viceversa for valleys!

If the method is interval/hybrid, then the intervals argument must be passed
and it should be an instance of Intervals class.

If the method is slope/hybrid, then the lookahead and avg_window
arguments should be changed based on the application. 
They have some default values though.

The method stores peaks in self.peaks in the following format:
{"peaks":[[peak positions], [peak amplitudes]],
"valleys": [[valley positions], [valley amplitudes]]}
```

In [ ]:
#things you can change
smooth_amount = 9
peak_amp_threshold = 3e-05
valley_amp_threshold = 2e-05

In [193]:
### Get peaks and valleys from data and make dataframes

hist = pypk.Data(data.index, data.values, smoothness=smooth_amount)
hist.get_peaks(method='slope', peak_amp_thresh=peak_amp_threshold, valley_thresh=valley_amp_threshold)
valleys, peaks = hist.peaks.items()

def flatten_list_of_iterables(iterable_of_iterables):
    """
    iterable_of_iterables is an iterable of single-value iterables
    
    some of the data is a list or array of single-value lists or arrays
    this flattens them by iterating thru the iterable of arrays/lists and append the 
    first (and only) element from each into a new 1-D array.
    """
    return np.array([iterable[0] for iterable in iterable_of_iterables])

vx, vy = valleys[1]
px, py = peaks[1]
vy = flatten_y_list(vy)
py = flatten_y_list(py)

peaks_df = pd.DataFrame(data=zip(px,py), columns=["t","A"]).sort('t')
valleys_df = pd.DataFrame(data=zip(vx,vy), columns=["t","A"]).sort('t')
valleys_df.to_csv('valleys.csv')
peaks_df.to_csv('peaks.csv')

In [198]:
list(data.index.values)[0:10]

[0.0,
 0.002,
 0.0040000000000000001,
 0.0060000000000000001,
 0.0080000000000000002,
 0.01,
 0.012,
 0.013999999999999999,
 0.016,
 0.018000000000000002]

In [200]:
combined.T

,0,10,20,31,19,100,103,32,74,95,...,45,46,35,81,14,11,29,30,29,62
t,0,0.184,0.244,0.246,0.348,0.486,0.488,0.552,0.644,0.646,...,18.838,18.84,19.018,19.14,19.586,19.646,19.698,19.7,19.878,19.996
A,-0.1147555,0.1000384,-0.01075612,-0.01106107,0.06783172,0.03620093,0.03615505,0.05649252,-0.009379879,-0.009704598,...,-0.03019017,-0.03048438,0.1842513,-0.03221145,-0.05304129,0.2252763,-0.05098415,-0.05137804,0.1737293,-0.03082221
type,v,p,v,v,p,v,v,p,v,v,...,v,v,p,v,v,p,v,v,p,v


In [216]:
data.head()

Elapsed time,iltered
(seconds),(mV)
0.000,-0.115
0.002,-0.115
0.004,-0.120
0.006,-0.120
0.008,-0.120


In [220]:
combined.head()

,t,A,type
0,0.000,-0.114756,v
10,0.184,0.100038,p
20,0.244,-0.010756,v
31,0.246,-0.011061,v
19,0.348,0.067832,p


In [264]:
peaks_df['type'] = 'p'
valleys_df['type'] = 'v'
combined = pd.concat([peaks_df,valleys_df]).sort('t')
#print combined.head()

event_times = []
orig_amps = []
types = []
for k,df in combined.T.iteritems():
    
    time = float(df['t'])
    event_times.append(time)
    
    amp = data.loc[time].values
    orig_amps.append(amp)
    
    feature_type = df['type']
    types.append(feature_type)
    
    #print k, ": ", time, amp, feature_type
    
amps = flatten_list_of_iterables(orig_amps)
temp_data = zip(event_times, amps, types)
index_list = range(len(event_times))
pv_orig_amps = pd.DataFrame(data = temp_data, index = index_list, columns=['t','A','type'])
pv_orig_amps.head(10)

,t,A,type
0,0.000,-0.115,v
1,0.184,0.125,p
2,0.244,-0.060,v
3,0.246,-0.065,v
4,0.348,0.075,p
5,0.486,0.020,v
6,0.488,0.025,v
7,0.552,0.055,p
8,0.644,-0.010,v
9,0.646,-0.010,v


In [265]:
#separate peaks from valleys
groupby_type = pv_orig_amps.groupby('type')
p, v = [groupby_type.get_group(i) for i in groupby_type.groups]
#print p
#now find R features using the np.histogram

print np.histogram(p['A'])
print
two_bin =  np.histogram(p['A'],2)
print two_bin
import sys
sys.stdout.flush()

plt.hist(p['A'].values,2)
plt.hist(p['A'].values)
plt.show()

(array([ 9, 16,  7,  2,  0,  2,  4, 12,  4,  2]), array([ 0.055 ,  0.1275,  0.2   ,  0.2725,  0.345 ,  0.4175,  0.49  ,
        0.5625,  0.635 ,  0.7075,  0.78  ]))

(array([34, 24]), array([ 0.055 ,  0.4175,  0.78  ]))


In [267]:
bin_edges = two_bin[1]
mid = bin_edges[1]
index_of_R_feats = pv_orig_amps[pv_orig_amps['A'] >= mid].index
index_of_T_feats = pv_orig_amps[pv_orig_amps['A'] <= mid].index
index_of_R_feats, index_of_T_feats

(Int64Index([ 10,  16,  25,  31,  40,  46,  55,  61,  67,  73,  79,  91,  97,
             103, 109, 115, 124, 130, 139, 145, 151, 157, 163, 169],
            dtype='int64'),
 Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9, 
             ...
             162, 164, 165, 166, 167, 168, 170, 171, 172, 173],
            dtype='int64', length=150))

####Requires installation of PyBursts (https://github.com/rpoddighe/pybursts)

In [150]:
import pybursts
#this doesn't group entire QRST complex, but it does seem to get the QRS complex pretty well (h =1)

bursts = pybursts.kleinberg(combined.t, s=2, gamma=.001)
#print bursts
#b = [i[1:] for i in bursts]
bursts_df =  pd.DataFrame(data = bursts, index = range(len(bursts)), columns=['h','start','end'])
print len(bursts_df)
bursts_df.head()

211


,h,start,end
0,0,0.002,19.996
1,1,0.184,0.246
2,2,0.244,0.246
3,3,0.244,0.246
4,4,0.244,0.246


In [160]:
ymin, ymax = plt.ylim()
maxh = max(bursts_df['h'])
sorted_bursts = bursts_df.sort("h")
"""
for r,c in zip(sorted_bursts.iterrows(),colors.cnames):#zip(bursts_df.start, bursts_df.end, colors.cnames):#bursts_df.iterrows():
    #print r
    h,s,e = r[1]
    #print h
    if h != 1:
        continue
    plt.vlines(s, ymin, ymax, color=c, lw=maxh-h)
    plt.vlines(e, ymin, ymax, color=c, lw=maxh-h)
    plt.hlines(ymin, s,e, color=c, lw=maxh-h)
    plt.hlines(ymax, s, e, color=c, lw=maxh-h)
    #break"""
plt.plot(data.index, data.values,'y', label = "original")
#plt.plot(hist.x,hist.y, "k",label="smooth", lw=2)
plt.plot(vx, vy ,"ro", label="valleys")
plt.plot(px, py, "co", label="peaks")

plt.xlim(5,10)
plt.legend()
plt.show()

In [15]:
import pybursts

offsets = [4, 17, 23, 27, 33, 35, 37, 76, 77, 82, 84, 88, 90, 92]
print pybursts.kleinberg(offsets, s=2, gamma=0.1)

[[0 4 92]
 [1.0 33 37]
 [1.0 76 92]
 [2.0 76 77]]


#Using BASS

In [1]:
#provide directory in which BASS is located
bass_folder_loc = '/Users/morganfine-morris/Documents/Repos/bass'
#uncomment '%matplotlib inline' below see plots inline (and remove blocking). 
#If you change your mind after running it, you must restart kernel
#%matplotlib inline 
#%matplotlib qt

import sys
import os
import os.path as path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# append the path to the folder containing bass 
# (which should prob. contain a __init__.py file.) to pythonpath
sys.path.append(bass_folder_loc) 
import bass
reload(bass)

BASS ready!
BASS ready!


<module 'bass' from '/Users/morganfine-morris/Documents/Repos/bass/bass.pyc'>

In [34]:
#initialize new file
Data = {}
Settings = {}
Results ={}

############################################################################################
#manual Setting block
Settings['folder']= r"./"#Physionet_ECGs"
Settings['Label'] = 'p01_r01_f.txt'#r'sel_100_tabsep.csv'
Settings['Output Folder'] = 'output'

#transformation Settings
Settings['Absolute Value'] = True #Must be True if Savitzky-Golay is being used

Settings['Bandpass Highcut'] =  'none'#in Hz
Settings['Bandpass Lowcut'] = 'none' #in Hz
Settings['Bandpass Polynomial'] = 'none' #integer

Settings['Linear Fit'] = False #between 0 and 1 on the whole time series
Settings['Linear Fit-Rolling R'] = 0.75 #between 0 and 1
Settings['Linear Fit-Rolling Window'] = 1000 #window for rolling mean for fit, unit is index not time
Settings['Relative Baseline'] = 0 #default 0, unless data is normalized, then 1.0. Can be any float

#integer
Settings['Savitzky-Golay Polynomial'] = 4 
#must be odd. units are index not time
Settings['Savitzky-Golay Window Size'] = 15 

#Baseline Settings
Settings['Baseline Type'] = r'rolling' #'linear', 'rolling', or 'static'
#For Linear
Settings['Baseline Start'] = 10.04 #start time in seconds
Settings['Baseline Stop'] = 10.18 #end time in seconds
#For Rolling
Settings['Rolling Baseline Window'] = 1 # in seconds. leave as 'none' if linear or static

#Peaks
Settings['Delta'] = .2#0.25
Settings['Peak Minimum'] = -1 #amplitude value
Settings['Peak Maximum'] = 1 #amplitude value

#Bursts
Settings['Burst Area'] = False #calculate burst area
Settings['Exclude Edges'] = False #False to keep edges, True to discard them
Settings['Inter-event interval minimum (seconds)'] = 0.00100 #only for bursts, not for peaks
Settings['Maximum Burst Duration (s)'] = 10 
Settings['Minimum Burst Duration (s)'] = 0
Settings['Minimum Peak Number'] = 1 #minimum number of peaks/burst, integer
#Settings['Threshold']= 4.9#.3 #linear: proportion of baseline. 
                           #static: literal value.
                           #rolling, linear ammount grater than rolling baseline at each time point.
if Settings['Baseline Type']=='rolling':
    Settings['Threshold']= .01 #linear: proportion of baseline. 
elif Settings['Baseline Type']=='static':
    Settings['Threshold']= 0.1#4.9 #linear: proportion of baseline. 
else:
    pass
#Outputs
Settings['Generate Graphs'] = False #create and save the fancy graph outputs


#Settings that you should not change unless you are a super advanced user:
#These are settings that are still in development
Settings['Graph LCpro events'] = False
Settings['File Type'] = r'Plain' #'LCPro', 'ImageJ', 'SIMA', 'Plain', 'Morgan'
Settings['Milliseconds'] = False
############################################################################################

In [35]:
#remove header from file and resave file (overwrites original data)
path_file = path.join(Settings['folder'], Settings['Label'])
file_data = pd.DataFrame.from_csv(path_file, header=0, sep="\t")

try:
    new = pd.DataFrame(file_data.values, file_data.index.values, dtype=float)
except:
    #its a string, remove it
    new = pd.DataFrame(file_data.values[1:], file_data.index.values[1:])

new.to_csv(path_file, sep='\t',header=False)
new.head()

,0
0.016,-0.115
0.018,-0.115
0.020,-0.110
0.022,-0.110
0.024,-0.110


#Data Processing (not file specific)

In [24]:
reload(bass)
Data, Settings = bass.load_wrapper({},Settings)
Data, Settings = bass.transform_wrapper(Data, Settings) #b/c I can't find smoothing settings that don't make it worse
#Data['trans'] = Data['original']
Data, Settings, Results = bass.baseline_wrapper(Data, Settings, Results)
#event detection
Results = bass.event_peakdet_wrapper(Data, Settings, Results)
Results = bass.event_burstdet_wrapper(Data, Settings, Results)

#clip the data from s seconds to e seconds
s=1
e=50
Data_clipped = {'original':Data['original'][s:e], 'trans':Data['trans'][s:e]}
#print Data_clipped
# graph_ts calls show, so unless you ran %matplotlib inline, 
# notebook will block until you close the plot window
#Data['original']['Mean2'][s:e].plot()
#plt.show()
axes = bass.graph_ts(Data_clipped, Settings, Results, roi="Mean1") 
plt.show()

BASS ready!
Made plots folder
Data Loaded
Sampling Rate= 0.002 (sec/frame)
p01_r01_f.txt is 19.988 seconds long.


In [192]:
#spectrogram
version = 'original'
key = 'Mean1'
bass.spectogram(version, key, Data, Settings, Results)
plt.show()

In [9]:
for key in ['Mean1']:
    #represent what a complete feature looks like for this particular ECG
    # s = burst start, e = burst end, v = valley, p=peak
    features_key = 'vspevspe'
    ''
    
    #count number of peaks and valleys in the features key
    peak_count = features_key.count('p')
    valley_count = features_key.count('v')

    
    peaks_df = Results['Peaks'][key].dropna() # R and T wave locations
    valleys_df = Results['Valleys'][key].dropna() # Q and S wave locations
    
    # make bursts_df which contains 'Burst Start' vals and 'Burst End' vals
    try:
        burst_ends = Results['Bursts'][key]['Burst End']
        burst_starts = Results['Bursts'][key]['Burst Start']
        bursts_df = pd.concat([burst_starts, burst_ends], axis=1)
        
        bursts_df.columns = ['Start','End']
    except:
        print "No bursts."
        
    #get peak and valley times
    print (peaks_df.head())
    peak_time = peaks_df.index.values
    valley_time = valleys_df.index.values
    
print peaks_df["Peaks Amplitude"].head()
n, pEdges = np.histogram(peaks_df["Peaks Amplitude"],bins=peak_count)
n, vEdges = np.histogram(valleys_df,bins=valley_count)
print pEdges, vEdges

       Peaks Amplitude  Intervals
Time                             
0.704            0.625      0.234
0.938            0.175      0.516
1.454            0.595      0.220
1.674            0.175      0.594
2.268            0.735      0.222
Time
0.704    0.625
0.938    0.175
1.454    0.595
1.674    0.175
2.268    0.735
Name: Peaks Amplitude, dtype: float64
0.175    4
0.620    4
0.210    3
0.550    3
0.165    3
0.650    2
0.195    2
0.610    2
0.575    1
0.615    1
0.690    1
0.150    1
0.285    1
0.595    1
0.495    1
0.180    1
0.490    1
0.600    1
0.160    1
0.670    1
0.245    1
0.655    1
0.215    1
0.200    1
0.795    1
0.185    1
0.480    1
0.225    1
0.325    1
0.735    1
0.190    1
0.625    1
dtype: int64
[ 0.15    0.4725  0.795 ] [-0.235 -0.135 -0.035]


#Getting Q, R, S, and T in the DataFrame and Ordered according to Time

Once you can see that Q, R, S, and T are being consistently detected (T should be marked by the burst ends) you can do the below process to label them. 

In [193]:
peaks_df = Results['Peaks']['Mean1'] # R wave location
valleys_df = Results['Valleys']['Mean1'] # Q and S wave locations
burst_ends = Results['Bursts']['Mean1']['Burst End'] #burst ends indicate approx. T wave location

# make new DataFrame for Burst Ends which is indexed by Burst Ends 
# b.c. Results['Bursts']['Mean1'] is indexed by Burst Start.
bursts_df = pd.DataFrame(burst_ends.values, index=burst_ends.values, columns=['Burst Ends'])

#combine peaks and valleys dfs and sort by time
# this will allow us to easily find peaks, 
# and find valleys before and after.
peaks_df['time'] = peaks_df.index
valleys_df['time'] = valleys_df.index
bursts_df['time'] = bursts_df.index
peaks_df['amplitude'] = peaks_df["Peaks Amplitude"]
valleys_df['amplitude'] = valleys_df["Valley Amplitude"]
bursts_df['amplitude'] = float(Settings['Threshold'])

#Label R and T. Unfortunatly, setting Q and S (the valleys) is much harder (See next cell).
peaks_df['letter'] = 'R'
bursts_df['letter'] = "T"

combined = peaks_df.append(valleys_df).sort('time')

#now combine w. Burst Ends and remove unnecessary columns, sort by time again
times = combined.append(bursts_df).drop([u'Burst Ends', u'Intervals', u'Peaks Amplitude', u'Valley Amplitude'],axis=1)
sorted_times = times.sort('time')
#remove na vals only from amplitude col.
sorted_times['amplitude'] = sorted_times['amplitude']
sorted_times.head()

KeyError: 'Mean1'

The following is a hack-y way to set the Q and S valleys, but it works. Later I might try to do it with booleans. <b>Feel free to try another way to do this!!! (Actually, really, please do. This method is pretty slow.)</b>

In [165]:
#reindex by integer
sorted_times['num'] = range(len(sorted_times))
sorted_times_reindexed = sorted_times.set_index('num')

letters = pd.DataFrame(index=sorted_times_reindexed.index, columns=['letters'], dtype=str)
for i in sorted_times_reindexed.index:
    
    #try to get the previous and next values with error handeling for edge cases
    try:
        prev_value = sorted_times_reindexed.loc[i-1, "letter"]
    except:
        prev_value=''
    try:
        next_value = sorted_times_reindexed.loc[i+1,"letter"]
    except:
        next_value=""
    
    #letters['time'][i] = sorted_times_reindexed.loc[i,'time']
    if (next_value == "T") and (prev_value == "R"):
        letters['letters'][i] = "S"
    elif next_value == "R":
        letters['letters'][i] = "Q"
    else:
        letters['letters'][i] = sorted_times_reindexed.loc[i,'letter']
        
sorted_times_reindexed['letter'] = letters['letters']
sorted_times_reindexed.head(13) #see that valleys which are not immediately before or after R are left as NaN!

,amplitude,letter,time
num,,,
0,-0.487,Q,0.12950
1,1.410,R,0.13475
2,-0.883,S,0.13825
3,0.030,T,0.15575
4,-0.455,Q,0.30125
5,1.514,R,0.30700
6,-0.934,S,0.31050
7,0.030,T,0.32575
8,-0.376,Q,0.47350


In [147]:
sorted_times_time_indexed = sorted_times_reindexed.set_index('time')
final = sorted_times_time_indexed.dropna()
final.head()

,amplitude,letter
time,,
0.12950,-0.487,Q
0.13475,1.410,R
0.13825,-0.883,S
0.15575,0.030,T
0.30125,-0.455,Q


In [153]:
s1=0
e1=20
final[s1:e1].plot()
plt.show()